In [1]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import display, SVG
from PIL import Image
import matplotlib.pyplot as plt

# Filepaths, numpy, pandas, and Tensorflow
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

# Keras
from keras.models import Sequential,Model
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D
from keras.preprocessing.image import ImageDataGenerator, load_img, image
from keras.applications import VGG16

# Shutil
import shutil

Using TensorFlow backend.


In [2]:
#t_directories = pd.read_csv('food-101/meta/train.txt', header=None)
#tList = t_directories[0].tolist()

#c_directories = pd.read_csv('food-101/meta/classes.txt', header=None)
#cList = c_directories[0].tolist()

#fList = [s for s in tList if any(xs in s for xs in cList)]

#with open('train.txt', 'w') as f:
#    for item in fList:
#        f.write("%s\n" % item)

In [8]:
#Only run the below code once in order to create the training directory

folderList = []

folder_directories = pd.read_csv('food-101/meta/classes.txt', header=None) #reads the classes and folders found in food-101
fList = folder_directories[0].tolist()

for item in fList:
    folderList.append(os.path.join('train/image',item)) #creates directory paths for folders needed

for item in folderList:
    os.mkdir(item)

In [6]:
#Only run the below code once in order to create the test directory

testD = []

test_directories = pd.read_csv('food-101/meta/classes.txt', header=None) #reads the classes and folders found in food-101
tfList = test_directories[0].tolist()

for item in tfList:
    testD.append(os.path.join('validation/image',item)) #creates directory paths for folders needed

for item in testD:
    os.mkdir(item)

In [9]:
currentD = []
nextD = []

t_directories = pd.read_csv('food-101/meta/train.txt', header=None)
tList = t_directories[0].tolist()

for item in tList: #only necessary to create training directory
    currentD.append(os.path.join('food-101/image',item + '.jpg'))
    nextD.append(os.path.join('train/image', item + '.jpg'))

for i in range(len(currentD)): #Only run this command once in order to create the training directory for images
    shutil.copy(currentD[i], nextD[i])

In [10]:
%%time
existingD = []
validationD = []

v_directories = pd.read_csv('food-101/meta/test.txt', header=None)
vList = v_directories[0].tolist()

for item in vList: #only necessary to create test directory
    existingD.append(os.path.join('food-101/image',item + '.jpg'))
    validationD.append(os.path.join('validation/image', item + '.jpg'))

for i in range(len(existingD)): #Only run this command once in order to create the training directory for images
    shutil.copy(existingD[i], validationD[i])

CPU times: user 770 ms, sys: 2.42 s, total: 3.19 s
Wall time: 6.06 s


In [11]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications import VGG16

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

vgg_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
# defining directories and amount of images to train/validate
train_dir = 'train/image'
validation_dir = 'validation/image'

nTrain = 16500
nVal = 5500

In [ ]:
%%time
# training images which belong to classes
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,22))

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

Found 16500 images belonging to 22 classes.


In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim= 7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(101, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size)

## Due to memory constraints, another notebook will be required in order to deal with Test data